# Домашнее задание 2 по обработке текстов

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная обработка данных

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [2]:
#Загружаем данные и закодируем мужской пол через 1, а женский через 0
female = pd.read_csv('female.txt', sep="\n", header = None, names = ['Name'])
male = pd.read_csv('male.txt', sep="\n", header = None, names = ['Name'])
female['Gender'] = 0
male['Gender'] = 1

In [3]:
#Удаляем неоднозначные данные и объединяем
female = female[~female['Name'].isin(male['Name'])]
male = male[~male['Name'].isin(female['Name'])]

##### Создание тестовой и сбалансированной обучающей выборки

In [4]:
len(male), len(female)

(2943, 4636)

Очевидно, что классы несбалансированы, для создания сбалансированной обучающей выборки сделаем downsampling класса женских имен.

In [5]:
female_downsampled = resample(female, 
                              replace=False,    
                              n_samples=len(male),     
                              random_state=123)
len(female_downsampled)

2943

Теперь классы сбалансированы.

In [6]:
data_balanced = pd.concat([male,female_downsampled], axis=0, ignore_index=True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data_balanced.Name, data_balanced.Gender, test_size=0.33, random_state=123)
print("Процент мужчин в обучающей выборке: {:.0f}%".format(sum(y_train) / len(y_train) * 100))
print("Процент мужчин в тестовой выборке: {:.0f}%".format(sum(y_test) / len(y_test) * 100)) 

Процент мужчин в обучающей выборке: 50%
Процент мужчин в тестовой выборке: 51%


Теперь обучающая и тестовая выборка сбалансированы, но мы можем добавить в тестовую выборку женские имена, которые выбросили ранее, чтобы сет для проверки был полноценным.

In [8]:
X_test = X_test.append(female['Name'][~female['Name'].isin(female_downsampled['Name'])])
y_test = y_test.append(female['Gender'][~female['Name'].isin(female_downsampled['Name'])])

In [9]:
print("Процент мужчин в обучающей выборке: {:.0f}%".format(sum(y_train) / len(y_train) * 100))
print("Процент мужчин в тестовой выборке: {:.0f}%".format(sum(y_test) / len(y_test) * 100))

Процент мужчин в обучающей выборке: 50%
Процент мужчин в тестовой выборке: 27%


##  Часть 2. Базовый метод классификации

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import *
from sklearn.metrics import *

##### Будем использовать логистическую регрессию, количество n-gram от 2 до 4

In [11]:
for n in range(2,5):
    clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(n,n), analyzer='char')),
    ('clf', LogisticRegression())])
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    print("For n = {}".format(n))
    print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions)))
    print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

For n = 2
F1-measure:   0.65
Accuracy:   0.78
For n = 3
F1-measure:   0.67
Accuracy:   0.78
For n = 4
F1-measure:   0.63
Accuracy:   0.73


Лучший результат по Accuracy и F1 показали 3-х символьные n-граммы

### Попробуем улучшить результаты:

In [12]:
female = pd.read_csv('female.txt', sep="\n", header = None, names = ['Name'])
male = pd.read_csv('male.txt', sep="\n", header = None, names = ['Name'])
female['Gender'] = 0
male['Gender'] = 1

Для этого используем более правильный алгоритм удаления неоднозначных имен:

In [13]:
#Удаляем неоднозначные данные и объединяем
ambiguos = female[female['Name'].isin(male['Name'])]
ambiguos = ambiguos.append(male[male['Name'].isin(female['Name'])], ignore_index=True)
male = male[~male['Name'].isin(ambiguos['Name'])]
female = female[~female['Name'].isin(ambiguos['Name'])]
len(male), len(female)

(2578, 4636)

Вместо downsamling'a класса женский имен сделаем апсемплинг класса мужских

In [14]:
male_upsampled = resample(male, 
                              replace=True,    
                              n_samples=len(female),     
                              random_state=123)
len(male_upsampled)

4636

Теперь классы сбалансированы.

In [15]:
data_balanced = pd.concat([male_upsampled,female], axis=0, ignore_index=True)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data_balanced.Name, data_balanced.Gender, test_size=0.33, random_state=123)
print("Процент мужчин в обучающей выборке: {:.0f}%".format(sum(y_train) / len(y_train) * 100))
print("Процент мужчин в тестовой выборке: {:.0f}%".format(sum(y_test) / len(y_test) * 100))

Процент мужчин в обучающей выборке: 50%
Процент мужчин в тестовой выборке: 50%


При обучении модели добавим нормализацию:

In [17]:
from sklearn.preprocessing import StandardScaler
for n in range(2,5):
    clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(n,n), analyzer='char')),
    ('normalizer', StandardScaler(with_mean=False)),    
    ('clf', LogisticRegression())])
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    print("For n = {}".format(n))
    print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions)))
    print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

For n = 2
F1-measure:   0.83
Accuracy:   0.83


C:\Users\adept\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\adept\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


For n = 3
F1-measure:   0.89
Accuracy:   0.89
For n = 4
F1-measure:   0.88
Accuracy:   0.87


C:\Users\adept\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


**Общий результат улучшился до 0.89 по обеим метрикам качества**